<a href="https://colab.research.google.com/github/DuncanFerguson/GITHUB_REPOSITORY/blob/main/Project_02_Duncan_Ferguson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [439]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import re

# Importing Sklearn libraries
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score

# nltk Libraries
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

!pip install pyprind
import pyprind

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Objectives
- 1). Read in a classifiable training datset from the Internet using Python for sentiment analysis

- 2). Understand the ethical implications of the dataset I've just identified, both in terms of the legality of its use, and in regards to any inherent biases present in the training data.

- 3). Preprocessing the training dataset to clean unwanted text data, and extract n-grams from the eample documents that can be used for model training.

- 4). Train a logistic regresion model using the cleaned training data that can be used to predict the sentiment of the unclassified examples

- 5). Use a grid search to optimize the logistic regression model.

- 6). Deploy the optimized and trained logistic regression model to the web using Python pickling, and provide a web form for users to further incrementally train your model by entering additional sample data and providing feedback about its classification accuracy.

# Experiment Objective

- Describe the dataset I am analyzing, including the TOS and ethical considerations discussed in Lecture 07 & 08 

The Data set that is being examined is from the University of California -Irvine's Machine Learning repository. The reason that I chose this data set is because it easy to come by. Last project I wasted two days looking for data sets. Now That I know UCI is easy to extract, I just chose one of their data sets so I could spend more time learning the code and less time data engineering a data set into google colab.

This sources is a well known public data set that is used for machine learning. By choosing to go with a public library I am not worring if I am violating a term of agreement. 

The Proper cite from the readme is below.
'This dataset was created for the Paper 'From Group to Individual Labels using Deep Features', Kotzias et. al,. KDD 2015'

https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences

- Why did I choose the training examples I did for training my model?

I chose the amazon data set for the training model for a few reasons. First there was an even amount of positive and negative reviews. Secondly, I feel as though amazon has a good imput for how different products are recieved. 


- How is sentiment determined for the training data?
# TODO


# Data Collection

- Am I accesing an API? Scaping a website? Downloading from an archive?

The data is being downloaded directly from an archive using !weget from google colab. This will grab the data straight from the source and place it straigt on your G-druve. URL below. https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences


- Write the necessary Python code to retrieve and store the training examples to create the model.
- The resulting implementation should be an in-memory pandas dataframe

In [440]:
# Grabbing the websit
!wget 'sentiment_labelled_sentences' 'https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip'

--2022-08-05 07:20:32--  http://sentiment_labelled_sentences/
Resolving sentiment_labelled_sentences (sentiment_labelled_sentences)... failed: Name or service not known.
wget: unable to resolve host address ‘sentiment_labelled_sentences’
--2022-08-05 07:20:32--  https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84188 (82K) [application/x-httpd-php]
Saving to: ‘sentiment labelled sentences.zip.10’

sentiment labelled  100%[===================>]  82.21K  --.-KB/s    in 0.02s   

2022-08-05 07:20:32 (3.80 MB/s) - ‘sentiment labelled sentences.zip.10’ saved [84188/84188]

FINISHED --2022-08-05 07:20:32--
Total wall clock time: 0.1s
Downloaded: 1 files, 82K in 0.02s (3.80 MB/s)


In [441]:
# Unzipping the file
!unzip -o 'sentiment labelled sentences.zip'

Archive:  sentiment labelled sentences.zip
  inflating: sentiment labelled sentences/.DS_Store  
  inflating: __MACOSX/sentiment labelled sentences/._.DS_Store  
  inflating: sentiment labelled sentences/amazon_cells_labelled.txt  
  inflating: sentiment labelled sentences/imdb_labelled.txt  
  inflating: __MACOSX/sentiment labelled sentences/._imdb_labelled.txt  
  inflating: sentiment labelled sentences/readme.txt  
  inflating: __MACOSX/sentiment labelled sentences/._readme.txt  
  inflating: sentiment labelled sentences/yelp_labelled.txt  
  inflating: __MACOSX/._sentiment labelled sentences  


In [442]:
# Displaying the data in a dataframe
df = pd.read_csv('sentiment labelled sentences/amazon_cells_labelled.txt', sep='\t', names=['review', 'Sentiment'])
# df2 = pd.read_csv('sentiment labelled sentences/imdb_labelled.txt', sep='\t', names=['review', 'Sentiment'])
df.head()
df_clean = df.copy()

In [500]:
# # Saving this nice little dataset as a an excel file
from google.colab import files
df.to_csv('amazon_sentiment.csv', encoding = 'utf-8', index=False)
files.download('amazon_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Data Preprocessing

- This is where the preprocessing utility for the project will be built.

- Steps required, clean text data, tokenize the document, construct a TfidfVectorizer to be performed here

In [444]:
# Using the Preprocessing of regular expresions from ch08
def preprocessor(text):
  """This preprocessing function removes all the punctuation, makes the word
  lower case, removews the stop words, and porterStems the words"""
  # Removing all the punctuation
  text = re.sub('<[^>]*>', '', text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
  text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')).strip()
  return text

# NOt that the tokenizing is set up for a pipeline run. 
def tokenizer(text):
  """Setting up the tokenizer ie split"""
  return text.split()

def tokenizer_porter(text):
  """Tokenizing and porting"""
  porter = PorterStemmer()
  return [porter.stem(word) for word in text.split()]

def tokenizer_stop(text):
  return [word for word in tokenizer_porter(text) if word not in stop]

stop = stopwords.words('english')
x = df['review'].apply(preprocessor)
y = df['Sentiment']


x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle=True)

# constructing a TfidfVectorizer
tfidf = TfidfVectorizer(use_idf=True, norm='l2', smooth_idf=True)

# Setting up the Param Grid
param_grid = [{'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter, tokenizer_stop],
               'clf__penalty': ['l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,tokenizer_porter, tokenizer_stop],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l2'],
               'clf__C': [1.0, 10.0, 100.0]}]

# Setting up the Pipeline
pipe = Pipeline(steps=[('vect', tfidf), 
                       ('clf', LogisticRegression(random_state=42,
                                                  solver='liblinear'))])

gs_pipe = GridSearchCV(estimator=pipe, param_grid=param_grid,
                       scoring='accuracy',
                       cv=5,
                       verbose=2,
                       n_jobs=-1,
                       error_score='raise')

gs_pipe.fit(x_train.to_list(), y_train.to_list())
print('Hyperparameters parameter set: %s ' % gs_pipe.best_params_)
print('Accuracy: %s ' % gs_pipe.best_score_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Hyperparameters parameter set: {'clf__C': 100.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__norm': None, 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x7f3cff361b90>, 'vect__use_idf': False} 
Accuracy: 0.8150000000000001 


# Model Optimization and Serialization

- This is where I will use a grid search similar to the one performed in lecture 11: Applying Machine Learning to Sentiment Anlysis

- Find the optimal hyperparameters (including choice of stemming algorithm for TfidVectorizer) to use with my out-of-core capable logistic regression classifier (like SGDClassifier)  Utilizing the TfidVectorizerI created in tepp three to preprocess my data

The Code for finding the hyperparameres and the Grid Search are right above

In [507]:
def stream_docs(path):
  with open(path, 'r', encoding='utf-8', newline='\n') as csv:
    next(csv) # skipping the header
    for line in csv:
      text, label = line[:-3], line[:-1][-1]
      yield text, label

display(next(stream_docs(path='amazon_sentiment.csv')))
# Displaying that Next Works
# for _ in range(10):
#   display(next(stream_docs(path='amazon_sentiment.csv')))

('So there is no way for me to plug it in here in the US unless I go by a converter.',
 '0')

('So there is no way for me to plug it in here in the US unless I go by a converter.',
 '0')

('So there is no way for me to plug it in here in the US unless I go by a converter.',
 '0')

('So there is no way for me to plug it in here in the US unless I go by a converter.',
 '0')

('So there is no way for me to plug it in here in the US unless I go by a converter.',
 '0')

('So there is no way for me to plug it in here in the US unless I go by a converter.',
 '0')

('So there is no way for me to plug it in here in the US unless I go by a converter.',
 '0')

('So there is no way for me to plug it in here in the US unless I go by a converter.',
 '0')

('So there is no way for me to plug it in here in the US unless I go by a converter.',
 '0')

('So there is no way for me to plug it in here in the US unless I go by a converter.',
 '0')

In [490]:
def get_minibatch(doc_stream, size):
  docs, y = [], []
  try:
    for _ in range(size):
      text, label = next(doc_stream)
      docs.append(text)
      y.append(label)
  except StopIteration:
    return None, None
  return docs, y

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=preprocessor,
                         tokenizer=tokenizer_stop)

clf = SGDClassifier(loss='log', random_state=42)

doc_stream = stream_docs(path='amazon_sentiment.csv')

In [494]:
pbar = pyprind.ProgBar(10)
classes = np.array([0,1])

for _ in range(45):
  x_train, y_train = get_minibatch(doc_stream, size=100) # Change size to be larger later on
  # if not x_train:
  #   break
  # display(y_train)
  # x_train = vect.transform(x_train)
  # clf.partial_fit(x_train, y_train.values.ravel(), classes=classes)
  # pbar.update()

None

None

None

None

None

...

None

None

None

None

None

In [492]:
x_test, y_test = get_minibatch(doc_stream, size=5000)
display(x_test)
# x_test = vect.transform(x_test)
# print('Accuracy: %.3f' % clf.score(x_test, y_test))

None


- Document the results of the grid seach in a marckdown cell immediately after the grid search, including computed accuracies agianst my training and test data sets.

- After documenting the finding fit the classifier against my entire dataset, then pickle the resulting object to a file that will be used in the website

* Note: Out-of-core learning is a requirement. Since I will be updating the model using the deployued Flask application, the classifier I pickle for yuse on the website must support incremental learning

* Note the hyperparamters I explore in the grid search must include different values than we used in class. One of the goals of this project is to see if other options migh yield better results than the book attempted. 

- Be sure to exaplain in the write up for this section why I chose the values that I did for the hyperparameters, and if they performed better for my data set than the other ones the booke attempted to use

* NOte: The grid search I perform will be agianst an something like an SGDClassifier instance, instead of the LogisticRegrssion classifier that was used in the book if you use SFDClassifer, instead of varying the value of C for regularization, you will vary the values for alpha. Still compare l1 vs l2 regulatization. This will result in a comparison of my work to the example in the book that isnt' strictly 1-1 but will be close enough to draw meaningful conculsions against


# Creat a standalone Python file

- File shoudl include all of the data preprocessing functionality created in step 4 named vectorizer.py. This file will initialize a HashingVectorizer using the parameters I learned tuning TfidVectorizer in step Four
- The creation of vectorizer.py should be done inside project02,pynb file similiar to ch09.ipynb

# Website Creation and Publishing

- Create an interactive website using Flask, SQlite, WTforms, and Jinja2, that will enable me to test the accuracy of my calculated model by prompting users to enter text in the same form as your train data (product reviews, social media posts, etc. ) and perform sentiment analysis against their submissionts

- Very similar to Lecture 12 -Embedding a Machine Learning Model into a Web Application, I will then show the user my sentiment prediction base on their entry, allowing them to rate whether the prediction was accurate or not. 

- User provided examples will be stored in SQLite and be used to incrementally refine the trained model I created in step 4. 

- Complete website will be a published to PthonAnywhere and a link to the website will be included in a markdown cell in this Jupter notebook.

- The source code for the website (including Python source code, HTML templates, and Python pickle files) will be submitted alongside the Jupyter Notbook in Goodle Drive 

- Note: I must use the pickled model created in step four as the starting point for my classifier. In addition, if your webserver was to be restared, all user-provided examples must still represet in the trained model (i.e. I cannot rest the pickled version only on server restart)

- Note: Make sure to look closely at the code thats being reused for the Flask application to make sure that assumptions built into the existing app still hold for the dataset. For instance, make sure that the handling of correct/incorrect feedback via the website works for my data.